In [1]:
import json
import gzip
import re
import pandas as pd
import numpy as np
#from sklearn.feature_extraction.text import TfidfVectorizer
from surprise import Reader, Dataset, accuracy, NormalPredictor, BaselineOnly, KNNBasic, SVD
#from sklearn.model_selection import train_test_split
#from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from surprise.model_selection import cross_validate, train_test_split, GridSearchCV

In [2]:
df_review = pd.read_json('Data/Video_Games_5.json.gz', compression='gzip', lines=True)

In [ ]:
#df_tg_review = pd.read_json('Data/Toys_and_Games_5.json.gz', compression='gzip', lines=True)

In [3]:
df_prod = pd.read_json('Data/meta_Video_Games.json.gz', compression='gzip', lines=True)

In [ ]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

df_tg_prod = getDF('Data/meta_Toys_and_Games.json.gz')

In [70]:
df_review.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,5,True,"10 17, 2015",A1HP7NVNPFMA4N,0700026657,Ambrosia075,"This game is a bit hard to get the hang of, bu...",but when you do it's great.,1445040000,NaN,NaN,NaN
1,4,False,"07 27, 2015",A1JGAP0185YJI6,0700026657,travis,I played it a while but it was alright. The st...,"But in spite of that it was fun, I liked it",1437955200,NaN,NaN,NaN
2,3,True,"02 23, 2015",A1YJWEXHQBWK2B,0700026657,Vincent G. Mezera,ok game.,Three Stars,1424649600,NaN,NaN,NaN
3,2,True,"02 20, 2015",A2204E1TH211HT,0700026657,Grandma KR,"found the game a bit too complicated, not what...",Two Stars,1424390400,NaN,NaN,NaN
4,5,True,"12 25, 2014",A2RF5B5H74JLPE,0700026657,jon,"great game, I love it and have played it since...",love this game,1419465600,NaN,NaN,NaN


In [ ]:
df_review.info()

In [3]:
df_prod.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,"[Video Games, PC, Games]",,[],,Reversi Sensory Challenger,[],,Fidelity Electronics,[],"[>#2,623,937 in Toys &amp; Games (See Top 100 ...",[],Toys &amp; Games,,,,0042000742,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
1,"[Video Games, Xbox 360, Games, </span></span><...",,[Brand new sealed!],,Medal of Honor: Warfighter - Includes Battlefi...,[B00PADROYW],,by\n \n EA Games,[],"[>#67,231 in Video Games (See Top 100 in Video...","[B0050SY5BM, B072NQJCW5, B000TI836G, B002SRSQ7...",Video Games,,,"\n\t\t\t\t\t\t\t\t\t\t\t\t<span class=""vertica...",0078764343,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
2,"[Video Games, Retro Gaming & Microconsoles, Su...",,[],,street fighter 2 II turbo super nintendo snes ...,[],,Nintendo,[],"[>#134,433 in Video Games (See Top 100 in Vide...",[],Video Games,,,$0.72,0276425316,[],[],NaN
3,"[Video Games, Xbox 360, Accessories, Controlle...",,[MAS's Pro Xbox 360 Stick (Perfect 360 Stick) ...,,Xbox 360 MAS STICK,[],,by\n \n MAS SYSTEMS,[Original PCB used from Xbox 360 Control Pad (...,"[>#105,263 in Video Games (See Top 100 in Vide...",[],Video Games,,,,0324411812,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
4,"[Video Games, PC, Games, </span></span></span>...",,"[Phonics Alive! 3, The Speller teaches student...",,Phonics Alive! 3: The Speller,[],,by\n \n Advanced Software Pty. Ltd.,"[Grades 2-12, Spelling Program, Teaches Spelli...","[>#92,397 in Video Games (See Top 100 in Video...",[B000BCZ7U0],Video Games,,,,0439335310,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN


In [4]:
df_prod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84819 entries, 0 to 84818
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   category         84819 non-null  object
 1   tech1            84819 non-null  object
 2   description      84819 non-null  object
 3   fit              84819 non-null  object
 4   title            84819 non-null  object
 5   also_buy         84819 non-null  object
 6   tech2            84819 non-null  object
 7   brand            84819 non-null  object
 8   feature          84819 non-null  object
 9   rank             84819 non-null  object
 10  also_view        84819 non-null  object
 11  main_cat         84819 non-null  object
 12  similar_item     84819 non-null  object
 13  date             84819 non-null  object
 14  price            84819 non-null  object
 15  asin             84819 non-null  object
 16  imageURL         84819 non-null  object
 17  imageURLHighRes  84819 non-null

In [4]:
df_prod.drop(['tech1', 'fit', 'tech2', 'date', 'price', 'imageURLHighRes', 'details'], axis=1, inplace=True)

In [54]:
#pd.set_option('display.max_colwidth', None)
df_prod.iloc[[1, -1]]

,category,description,title,also_buy,brand,feature,rank,also_view,main_cat,similar_item,asin,imageURL
1,"[Video Games, Xbox 360, Games, </span></span><...",[Brand new sealed!],Medal of Honor: Warfighter - Includes Battlefi...,[B00PADROYW],by\n \n EA Games,[],"[>#67,231 in Video Games (See Top 100 in Video...","[B0050SY5BM, B072NQJCW5, B000TI836G, B002SRSQ7...",Video Games,,0078764343,[https://images-na.ssl-images-amazon.com/image...
84818,"[Video Games, PC, Accessories, Gaming Mice]",[<b>Features and Functions:</b> <br>Quick thum...,Zelotes 4000 DPI Programmable Gaming Mouse for...,[],Zelotes,"[Customize DPI Switch: 1000/1500/2000/4000,the...",[>#434 in Computers & Accessories > Computer A...,"[B016ZF77OA, B00GU8W5AE, B00E64DTDQ, B074TF6XV...",Computers,,B01HJEBIAA,[https://images-na.ssl-images-amazon.com/image...


In [55]:
df_prod.head(20)

,category,description,title,also_buy,brand,feature,rank,also_view,main_cat,similar_item,asin,imageURL
0,"[Video Games, PC, Games]",[],Reversi Sensory Challenger,[],Fidelity Electronics,[],"[>#2,623,937 in Toys &amp; Games (See Top 100 ...",[],Toys &amp; Games,,0042000742,[https://images-na.ssl-images-amazon.com/image...
1,"[Video Games, Xbox 360, Games, </span></span><...",[Brand new sealed!],Medal of Honor: Warfighter - Includes Battlefi...,[B00PADROYW],by\n \n EA Games,[],"[>#67,231 in Video Games (See Top 100 in Video...","[B0050SY5BM, B072NQJCW5, B000TI836G, B002SRSQ7...",Video Games,,0078764343,[https://images-na.ssl-images-amazon.com/image...
2,"[Video Games, Retro Gaming & Microconsoles, Su...",[],street fighter 2 II turbo super nintendo snes ...,[],Nintendo,[],"[>#134,433 in Video Games (See Top 100 in Vide...",[],Video Games,,0276425316,[]
3,"[Video Games, Xbox 360, Accessories, Controlle...",[MAS's Pro Xbox 360 Stick (Perfect 360 Stick) ...,Xbox 360 MAS STICK,[],by\n \n MAS SYSTEMS,[Original PCB used from Xbox 360 Control Pad (...,"[>#105,263 in Video Games (See Top 100 in Vide...",[],Video Games,,0324411812,[https://images-na.ssl-images-amazon.com/image...
4,"[Video Games, PC, Games, </span></span></span>...","[Phonics Alive! 3, The Speller teaches student...",Phonics Alive! 3: The Speller,[],by\n \n Advanced Software Pty. Ltd.,"[Grades 2-12, Spelling Program, Teaches Spelli...","[>#92,397 in Video Games (See Top 100 in Video...",[B000BCZ7U0],Video Games,,0439335310,[https://images-na.ssl-images-amazon.com/image...
5,"[Video Games, PC, Games, </span></span></span>...",[CD-ROM],Sim City 3000,[],by\n \n Scholastic,[Sim City 3000 CD-ROM],"[>#19,087 in Video Games (See Top 100 in Video...","[B00004T1M7, B00000G16I, B00006GF1U, B0000C0YW...",Video Games,,0439339006,[https://images-na.ssl-images-amazon.com/image...
6,"[Video Games, PC, Games, </span></span></span>...",[],Phonics Alive! 2 The Sound Blender,[],by\n \n Advanced Software Party,[Phonics Alive! 2 The Sound Blender is a new a...,"[>#117,477 in Video Games (See Top 100 in Vide...",[],Video Games,,0439335299,[]
7,"[Video Games, PC, Games, </span></span></span>]",[a scholastic clubs fairs cd rom game],A to Zap,[],by\n \n sunburst,[],"[>#87,937 in Video Games (See Top 100 in Video...",[],Video Games,,0439339960,[https://images-na.ssl-images-amazon.com/image...
8,"[Video Games, PC, Games]",[video game],Need for Speed Porsche Unleashed,[],Electronic Arts,[video game],"[>#51,505 in Video Games (See Top 100 in Video...","[B00004DITL, B000067NP1, B00003W869]",Video Games,,043933702X,[https://images-na.ssl-images-amazon.com/image...
9,"[Video Games, PC, Games, </span></span></span>...",[Grandma Groupers kelp seeds are missing and w...,Freddi Fish and the Case of The Missing Kelp S...,"[B00002S6E5, B000NQI666, B000AQJ4AG, B000FETCF...",by\n \n Humongous Entertainment,"[Like the real world, Freddi Fishs world chang...","[>#58,523 in Video Games (See Top 100 in Video...","[B000FETCF0, B00005BU3M, B00004UCUT, B0009RAVF...",Video Games,,0439339987,[https://images-na.ssl-images-amazon.com/image...


In [5]:
df_prod['category'] = [','.join(map(str, l)) for l in df_prod['category']]

In [28]:
re.sub('</span>', '', df_prod['category'][1])

'Video Games,Xbox 360,Games,'

In [6]:
def remove_tags(string):
    exclusionlist = ['</span>','<b>', '</b>', '<br />', '\n', '/n', '</div>', "'<div>'", '<DIV>', '</DIV>']
    exclusions = '|'.join(exclusionlist)
    result = re.sub(exclusions, '', string)
    return result

In [7]:
df_prod['category'] = df_prod['category'].apply(lambda x : remove_tags(x))

In [8]:
df_prod['description'] = [''.join(map(str, l)) for l in df_prod['category']]

In [9]:
df_prod['description'] = df_prod['description'].apply(lambda x: remove_tags(x))

In [10]:
df_prod['brand'] = df_prod['brand'].apply(lambda x: re.sub('by', '', x))
df_prod['brand'] = df_prod['brand'].apply(lambda x: re.sub('\n', '', x))

In [66]:
df_prod

,category,description,title,also_buy,brand,feature,rank,also_view,main_cat,similar_item,asin,imageURL
0,"Video Games,PC,Games","Video Games,PC,Games",Reversi Sensory Challenger,[],Fidelity Electronics,[],"[>#2,623,937 in Toys &amp; Games (See Top 100 ...",[],Toys &amp; Games,,0042000742,[https://images-na.ssl-images-amazon.com/image...
1,"Video Games,Xbox 360,Games,","Video Games,Xbox 360,Games,",Medal of Honor: Warfighter - Includes Battlefi...,[B00PADROYW],EA Games,[],"[>#67,231 in Video Games (See Top 100 in Video...","[B0050SY5BM, B072NQJCW5, B000TI836G, B002SRSQ7...",Video Games,,0078764343,[https://images-na.ssl-images-amazon.com/image...
2,"Video Games,Retro Gaming & Microconsoles,Super...","Video Games,Retro Gaming & Microconsoles,Super...",street fighter 2 II turbo super nintendo snes ...,[],Nintendo,[],"[>#134,433 in Video Games (See Top 100 in Vide...",[],Video Games,,0276425316,[]
3,"Video Games,Xbox 360,Accessories,Controllers,J...","Video Games,Xbox 360,Accessories,Controllers,J...",Xbox 360 MAS STICK,[],MAS SYSTEMS,[Original PCB used from Xbox 360 Control Pad (...,"[>#105,263 in Video Games (See Top 100 in Vide...",[],Video Games,,0324411812,[https://images-na.ssl-images-amazon.com/image...
4,"Video Games,PC,Games,,Grades 2-12,Spelling Pro...","Video Games,PC,Games,,Grades 2-12,Spelling Pro...",Phonics Alive! 3: The Speller,[],Advanced Software Pty. Ltd.,"[Grades 2-12, Spelling Program, Teaches Spelli...","[>#92,397 in Video Games (See Top 100 in Video...",[B000BCZ7U0],Video Games,,0439335310,[https://images-na.ssl-images-amazon.com/image...
...,...,...,...,...,...,...,...,...,...,...,...,...
84814,"Video Games,PlayStation Vita,Digital Games & D...","Video Games,PlayStation Vita,Digital Games & D...",Lego Star Wars: The Force Awakens - PS Vita [D...,[],Warner Bros.,[],"[>#74,224 in Video Games (See Top 100 in Video...",[],Video Games,,B01HJ1521Y,[]
84815,"Video Games,PlayStation 4,Digital Games & DLC,...","Video Games,PlayStation 4,Digital Games & DLC,...",Lego Star Wars: The Force Awakens Season Pass...,[],Warner Bros.,[DLC Requires base game],"[>#62,150 in Video Games (See Top 100 in Video...",[],Video Games,,B01HJ14TTA,[]
84816,"Video Games,PlayStation 4,Digital Games & DLC,...","Video Games,PlayStation 4,Digital Games & DLC,...",The Technomancer - PS4 [Digital Code],[],Focus Home Interactive,[],"[>#94,234 in Video Games (See Top 100 in Video...",[],Video Games,,B01HJ14OT0,[https://images-na.ssl-images-amazon.com/image...
84817,"Video Games,Xbox 360,Accessories,","Video Games,Xbox 360,Accessories,","Repair T8 T6 Tools for XBOX One Xbox 360, YTTL...","[B01KBNB7K2, B06X6JSYPC, B01N6Y0Z7W, B06VXD2W5...",YTTL,[If you want to Replacement you Xbox one /360 ...,"[>#16,087 in Video Games (See Top 100 in Video...","[B01KH25ZY6, B00PG8SU26, B07G122BVS, B016XLTQP...",Video Games,,B01HJC33WS,[https://images-na.ssl-images-amazon.com/image...


In [118]:
df_prod['brand'].value_counts()

                           4027
        Electronic Arts    2612
        Activision         2292
        Nintendo           1931
        Sega               1569
                           ... 
        sunshine              1
        WS net                1
PrimeTime Products            1
        colorfull-box         1
Fluenz                        1
Name: brand, Length: 9196, dtype: int64

In [120]:
df_prod.loc[df_prod['brand']=='Nintendo']

,category,description,title,also_buy,brand,feature,rank,also_view,main_cat,similar_item,asin,imageURL
2,"Video Games,Retro Gaming & Microconsoles,Super...","Video Games,Retro Gaming & Microconsoles,Super...",street fighter 2 II turbo super nintendo snes ...,[],Nintendo,[],"[>#134,433 in Video Games (See Top 100 in Vide...",[],Video Games,,0276425316,[]
215,"Video Games,Retro Gaming & Microconsoles,Ninte...","Video Games,Retro Gaming & Microconsoles,Ninte...",Forsaken,"[B00002STGL, B00002STGQ, B00000JMWB, B00000DMB...",Nintendo,[Nintendo],"[>#43,237 in Video Games (See Top 100 in Video...","[B00002STGL, B00000DMAT, B00000DMB3, B00000I1B...",Video Games,,B000006OWT,[]
247,"Video Games,Retro Gaming & Microconsoles,Ninte...","Video Games,Retro Gaming & Microconsoles,Ninte...",1080 Snowboarding,"[B00002STI2, B00001QES1, B000021Y2Z, B00002STH...",Nintendo,[],"[>#7,719 in Video Games (See Top 100 in Video ...","[B00001QES1, B00002STI2, B00000DMAX, B00000F1G...",Video Games,,B00000DMAO,[https://images-na.ssl-images-amazon.com/image...
249,"Video Games,Retro Gaming & Microconsoles,Ninte...","Video Games,Retro Gaming & Microconsoles,Ninte...",Mario Kart 64,"[B00002DHEV, B00000F1GM, B0792NTVZ4, B0793J5C1...",Nintendo,[],[>#891 in Video Games (See Top 100 in Video Ga...,"[B00000F1GM, B00002DHEV, B00004VX12, B00000J2W...",Video Games,,B00000DMAX,[https://images-na.ssl-images-amazon.com/image...
251,"Video Games,Retro Gaming & Microconsoles,Ninte...","Video Games,Retro Gaming & Microconsoles,Ninte...",The Legend of Zelda: Ocarina of Time,"[B00000F1GM, B00002DHEV, B00000DMAX, B00002STE...",Nintendo,[],"[>#1,847 in Video Games (See Top 100 in Video ...","[B00000ID61, B00OVJQD4I, B000JVM256, B00004U1R...",Video Games,,B00000DMB3,[https://images-na.ssl-images-amazon.com/image...
...,...,...,...,...,...,...,...,...,...,...,...,...
84318,"Video Games,Nintendo 3DS & 2DS,Accessories","Video Games,Nintendo 3DS & 2DS,Accessories",Papi&nbsp;- Nintendo Animal Crossing Happy Hom...,"[B01CKH0XHA, B017W17G6K, B012F20ZU0, B01AXGBJP...",Nintendo,[Amiibo Animal Crossing Happy Home Designer Ca...,"[>#74,296 in Video Games (See Top 100 in Video...",[],Video Games,,B01GU9NOO8,[]
84320,"Video Games,Nintendo 3DS & 2DS,Accessories","Video Games,Nintendo 3DS & 2DS,Accessories",Ribbot&nbsp;- Nintendo Animal Crossing Happy H...,"[B01CKH0XHA, B01AVLWBY0, B01B1X6JBI]",Nintendo,[Amiibo Animal Crossing Happy Home Designer Ca...,"[>#71,091 in Video Games (See Top 100 in Video...",[],Video Games,,B01GU9NV0A,[]
84321,"Video Games,Nintendo 3DS & 2DS,Accessories","Video Games,Nintendo 3DS & 2DS,Accessories",Gloria&nbsp;- Nintendo Animal Crossing Happy H...,[],Nintendo,[Amiibo Animal Crossing Happy Home Designer Ca...,"[>#103,901 in Video Games (See Top 100 in Vide...",[],Video Games,,B01GU9NOPC,[]
84671,"Video Games,Kids & Family,Nintendo 3DS,Games","Video Games,Kids & Family,Nintendo 3DS,Games",Beyblade 3DS Japan Import,"[B07JCM53Y5, B07D3J3HJ9, B06Y6LNFRZ, B07H9L6LH...",Nintendo,[],"[>#7,410 in Video Games (See Top 100 in Video ...","[B00CP1QC4K, B00CP1Q9LQ, B0732S9YGW, B07D3J3HJ...",Video Games,,B01HBAUF56,[https://images-na.ssl-images-amazon.com/image...


## Content Based Recommendation System

In [117]:
train_df1, test_df1 = train_test_split(df_prod, random_state=42)

AttributeError: 'DataFrame' object has no attribute 'raw_ratings'

In [79]:
train_df

,category,description,title,also_buy,brand,feature,rank,also_view,main_cat,similar_item,asin,imageURL
34661,"Video Games,Wii,Accessories,Mounts, Brackets &...","Video Games,Wii,Accessories,Mounts, Brackets &...",Chill Station for Wii,[],Psyclone Games,"[Illuminated Base, USB Powered, Prevents Knock...","[>#209,907 in Video Games (See Top 100 in Vide...",[],Video Games,,B000P4RMO8,[https://images-na.ssl-images-amazon.com/image...
24753,"Video Games,PC,Games,,fast paced singleplayer ...","Video Games,PC,Games,,fast paced singleplayer ...",Mafia (Jewel Case) - PC,[B001Q96ZNU],Global Star Software,[fast paced singleplayer shooter game.],"[>#63,423 in Video Games (See Top 100 in Video...",[B00004U8K2],Video Games,,B0001PK3WE,[]
84389,"Video Games,Digital Games,PC,,Delivers an expe...","Video Games,Digital Games,PC,,Delivers an expe...",Resident Evil 7: Biohazard - PS4 Digital Code,"[B07G9WK8GS, B004RMK4BC, B004RMK4P8, B00GAC1D2...",Capcom,[Delivers an experience reminiscent of the ser...,[],"[B07DK3PNWQ, B07JQRWWKR, B07HKRRRZ9, B072LXFN8...",Video Games,,B01GW3POY0,[https://images-na.ssl-images-amazon.com/image...
80310,"Video Games,PlayStation 3,Accessories,","Video Games,PlayStation 3,Accessories,",Brook PC PS3 PS4 Fight Board Fighting DIY Kit ...,"[B0713PGNHD, B01C5J5AJO, B005BZ421M]",Brook,[Plug and play. Multi-console support: Compati...,"[>#35,925 in Video Games (See Top 100 in Video...","[B0713PGNHD, B005BIC9QE, B07BYCGW77, B00UUROWW...",Video Games,,B01A29YQ1M,[https://images-na.ssl-images-amazon.com/image...
50353,"Video Games,PlayStation 3,Games,","Video Games,PlayStation 3,Games,",BEYOND: Two Souls,"[B005YR308S, B007CM0K86, B008XMSR6G, B002CZ38K...",Sony,[Dive into a gripping and unpredictable psycho...,"[>#2,756 in Video Games (See Top 100 in Video ...","[B007CM0K86, B01C90JRZW, B004CDBP9G, B005YR308...",Video Games,,B0050SX7BQ,[https://images-na.ssl-images-amazon.com/image...
...,...,...,...,...,...,...,...,...,...,...,...,...
6265,"Video Games,Retro Gaming & Microconsoles,Game ...","Video Games,Retro Gaming & Microconsoles,Game ...",Wormlight Advance: Purple - Game Boy Advance,[B00005B8G3],Nyko,[],"[>#75,963 in Video Games (See Top 100 in Video...","[B01GB3CIEA, B004910A8I, B01GAX38OA, B001DL8PE...",Video Games,,B00005JYBA,[]
54886,"Video Games,PC,Games,,After 20 years of best-s...","Video Games,PC,Games,,After 20 years of best-s...",The Elder Scrolls Online - PC/Mac,"[B01N39FFYS, B07BJ5Q293, B00STU8L3Q, B00CJ7IUI...",Bethesda,"[After 20 years of best-selling, award-winning...","[>#18,920 in Video Games (See Top 100 in Video...","[B01N39FFYS, B00STU8L3Q, B07BJ5Q293, B01I71VGP...",Video Games,,B0081Q58AW,[https://images-na.ssl-images-amazon.com/image...
76820,"Video Games,Nintendo 3DS & 2DS,Accessories,Fac...","Video Games,Nintendo 3DS & 2DS,Accessories,Fac...",Omega Ruby Alpha Sapphire Primal Groudon Video...,[],,[],"[>#88,811 in Video Games (See Top 100 in Video...",[],Video Games,,B0145KQZ5Y,[https://images-na.ssl-images-amazon.com/image...
860,"Video Games,PC,Games,","Video Games,PC,Games,",Prince of Persia 3-D - PC,[],The Learning Company,[],"[>#94,050 in Video Games (See Top 100 in Video...",[B00005YX8F],Video Games,,B00001W0HL,[https://images-na.ssl-images-amazon.com/image...


In [116]:
train_df['brand'].value_counts()

TypeError: 'Trainset' object is not subscriptable

In [91]:
# description based
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(train_df['description'])

In [92]:
tfidf_matrix = tfidf_matrix.astype(np.float32)
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

RuntimeError: nnz of the result is too large

## Collaborative

In [11]:
df_review.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,5,True,"10 17, 2015",A1HP7NVNPFMA4N,0700026657,Ambrosia075,"This game is a bit hard to get the hang of, bu...",but when you do it's great.,1445040000,NaN,NaN,NaN
1,4,False,"07 27, 2015",A1JGAP0185YJI6,0700026657,travis,I played it a while but it was alright. The st...,"But in spite of that it was fun, I liked it",1437955200,NaN,NaN,NaN
2,3,True,"02 23, 2015",A1YJWEXHQBWK2B,0700026657,Vincent G. Mezera,ok game.,Three Stars,1424649600,NaN,NaN,NaN
3,2,True,"02 20, 2015",A2204E1TH211HT,0700026657,Grandma KR,"found the game a bit too complicated, not what...",Two Stars,1424390400,NaN,NaN,NaN
4,5,True,"12 25, 2014",A2RF5B5H74JLPE,0700026657,jon,"great game, I love it and have played it since...",love this game,1419465600,NaN,NaN,NaN


In [100]:
df_review

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,5,True,"10 17, 2015",A1HP7NVNPFMA4N,0700026657,Ambrosia075,"This game is a bit hard to get the hang of, bu...",but when you do it's great.,1445040000,NaN,,NaN
1,4,False,"07 27, 2015",A1JGAP0185YJI6,0700026657,travis,I played it a while but it was alright. The st...,"But in spite of that it was fun, I liked it",1437955200,NaN,,NaN
2,3,True,"02 23, 2015",A1YJWEXHQBWK2B,0700026657,Vincent G. Mezera,ok game.,Three Stars,1424649600,NaN,,NaN
3,2,True,"02 20, 2015",A2204E1TH211HT,0700026657,Grandma KR,"found the game a bit too complicated, not what...",Two Stars,1424390400,NaN,,NaN
4,5,True,"12 25, 2014",A2RF5B5H74JLPE,0700026657,jon,"great game, I love it and have played it since...",love this game,1419465600,NaN,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
497572,4,True,"07 1, 2017",AVECM71LSZLC5,B01HGPUTCA,boris teplitskiy,not OEM but good replacement parts,Four Stars,1498867200,NaN,,NaN
497573,3,True,"08 20, 2018",A1RS06313BL6WN,B01HH6JEOC,Tom Stopsign,Okay stuff.,Three Stars,1534723200,NaN,"{'Edition:': ' Kids Room', 'Platform:': ' PC O...",NaN
497574,3,True,"08 7, 2017",ACIZ77IGIX2JL,B01HH6JEOC,Era,This does add some kids room things that are v...,Only buy on sale.,1502064000,NaN,"{'Edition:': ' Kids Room', 'Platform:': ' PC O...",NaN
497575,4,False,"08 5, 2018",A34GG58TJ1A3SH,B01HIZF7XE,seamonkey10,I think I originally began playing Bioshock se...,"It's Okay, Nothing Profound",1533427200,NaN,"{'Edition:': ' Collection', 'Platform:': ' Xbo...",NaN


In [12]:
df_review.drop(['verified', 'reviewTime', 'reviewerName', 'unixReviewTime', 'vote', 'image'], axis=1, inplace=True)

In [13]:
df_review.head()

,overall,reviewerID,asin,reviewText,summary,style
0,5,A1HP7NVNPFMA4N,0700026657,"This game is a bit hard to get the hang of, bu...",but when you do it's great.,NaN
1,4,A1JGAP0185YJI6,0700026657,I played it a while but it was alright. The st...,"But in spite of that it was fun, I liked it",NaN
2,3,A1YJWEXHQBWK2B,0700026657,ok game.,Three Stars,NaN
3,2,A2204E1TH211HT,0700026657,"found the game a bit too complicated, not what...",Two Stars,NaN
4,5,A2RF5B5H74JLPE,0700026657,"great game, I love it and have played it since...",love this game,NaN


In [14]:
df_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497577 entries, 0 to 497576
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   overall     497577 non-null  int64 
 1   reviewerID  497577 non-null  object
 2   asin        497577 non-null  object
 3   reviewText  497419 non-null  object
 4   summary     497468 non-null  object
 5   style       289237 non-null  object
dtypes: int64(1), object(5)
memory usage: 22.8+ MB


In [15]:
df_review['style'].fillna(value='', inplace=True)
df_review['reviewText'].fillna(value='', inplace=True)
df_review['summary'].fillna(value='', inplace=True)

In [16]:
surprise_df = df_review[[ 'reviewerID', 'asin', 'overall']]

In [17]:
surprise_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497577 entries, 0 to 497576
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   reviewerID  497577 non-null  object
 1   asin        497577 non-null  object
 2   overall     497577 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 11.4+ MB


In [18]:
read = Reader(rating_scale=(1, 5))
surprise_data = Dataset.load_from_df(surprise_df, read)
train_df, test_df = train_test_split(surprise_data, random_state=42)

### Baseline

In [19]:
baseline_model1 = NormalPredictor()
baseline_model1.fit(train_df)

In [20]:
prediction1 = baseline_model1.test(test_df)

In [21]:
accuracy.rmse(prediction1)

RMSE: 1.5169


1.5168761365568832

In [22]:
baseline_model2 = BaselineOnly()
baseline_model2.fit(train_df)

Estimating biases using als...


In [23]:
predictions2 = baseline_model2.test(test_df)

In [24]:
accuracy.rmse(predictions2)

RMSE: 1.0447


1.0446509835320532

### SVD

In [25]:
svd = SVD(random_state=42)
svd.fit(train_df)
prediction_svd = svd.test(test_df)
accuracy.rmse(prediction_svd)

RMSE: 1.0152


1.0151596982698952